In [ ]:
import serial
from facenet_pytorch import MTCNN
import torch
import numpy as np
import cv2
from PIL import Image, ImageDraw
import numpy as np
from tensorflow.keras import models
from recording_helper import record_audio, terminate
from tf_helper import preprocess_audiobuffer
import time
import pyttsx3
import win32com.client


engine = pyttsx3.init()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))
mtcnn = MTCNN(keep_all=True, device=device)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30) # set the frame rate to 30 frames per second
frames = []
frames_tracked = []

last_face_detected = time.time()


commands = ['noise', 'noise', 'noise', 'noise', 'right', 'noise', 'up', 'noise']

loaded_model = models.load_model("saved_model")

def predict_mic():
    audio = record_audio()
    spec = preprocess_audiobuffer(audio)
    prediction = loaded_model(spec)
    label_pred = np.argmax(prediction, axis=1)
    command = commands[label_pred[0]]
    print("Predicted label:", command)
    return command

ser = serial.Serial('COM3', 9600)  # change the port to match the port of the arduino

while True:  # main loop
    ret, frame = cap.read()
    if not ret:
        break
    frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    frames.append(frame)

    # Detect faces
    boxes, _ = mtcnn.detect(frame)
    
    # Draw faces
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)
    if boxes is not None:
        for box in boxes:
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
            print("Welcome! Please say the password.")
            engine.say("Welcome! Please say the password.")
            engine.runAndWait()
        password_input = []
        for i in range(5):
            password_input.append(predict_mic())
            time.sleep(0.005)

        right_count = 0
        up_count = 0
        for i in range(len(password_input)):
            if password_input[i] == "right":
                right_count += 1
                if right_count == 2 and up_count == 1:
                    print("Access granted. The door will open")
                    engine.say("Access granted. The door will open")
                    engine.runAndWait()
                    ser.write(b'1')  # send '1' to activate the servo motor
                    time.sleep(3)  # wait for 3 seconds
                    ser.write(b'0')  # send '0' to deactivate the servo motor
                    break
            elif password_input[i] == "up":
                up_count += 1
                if right_count >= 2 and up_count >= 1:
                    print("Access granted. The door will open")
                    engine.say("Access granted. The door will open")
                    engine.runAndWait()
                    ser.write(b'1')  # send '1' to activate the servo motor
                    time.sleep(3)  # wait for 3 seconds
                    ser.write(b'0')  # send '0' to deactivate the servo motor
                    time.sleep(5) # added delay
                    break
        else:
            print("Access denied.")
            engine.say("Access denied.")
            engine.runAndWait()
            time.sleep(5) # added delay
    # Add to frame list
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))

    cv2.imshow("Webcam Face Detection", cv2.cvtColor(np.array(frame_draw), cv2.COLOR_RGB2BGR))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

cap.release()
cv2.destroyAllWindows()

Running on device: cpu
Welcome! Please say the password.
Predicted label: right
Predicted label: right
Predicted label: up
Predicted label: noise
Predicted label: noise
Access granted. The door will open
Welcome! Please say the password.
Predicted label: up
Predicted label: up
Predicted label: noise
Predicted label: noise
Predicted label: up
Access denied.
Welcome! Please say the password.
Predicted label: right
Predicted label: right
Predicted label: up
Predicted label: noise
Predicted label: noise
Access granted. The door will open
Welcome! Please say the password.
Predicted label: noise
Predicted label: noise
Predicted label: noise
Predicted label: noise
Predicted label: noise
Access denied.
